In [2]:
import pandas as pd
from ultralytics import YOLO

In [3]:
augmented_model_path = "./augmented.pt" # Local path where manually downloaded model from GCS is saved
test_results_path = "./test_results"  # Local folder for test results
test_data_yaml = "./test_data.yaml"  # Temporary YAML file for test data configuration

In [4]:
# Initialize augmented YOLO model
print("[INFO] Loading YOLO model...")
model = YOLO(augmented_model_path)

[INFO] Loading YOLO model...


In [5]:
model.val(
    data=test_data_yaml,  # Use the YAML file for test data
    save_json=True,       # Save results in JSON format
    save_conf=True,       # Save confidence scores
    project=test_results_path, # Save results in the specified folder
    name="augmented_test_predictions",
    plots=True            # Enable plotting for visualization (can disable if not needed)
    )

# Display validation metrics from YOLO output
print("\nValidation completed. Check the metrics above and JSON file for detailed results.")

Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:13


                   all        163        878      0.759      0.838      0.864      0.729
                     V        162        694      0.815      0.867      0.912      0.733
                     C         79        100      0.582      0.767      0.747      0.644
                     S         64         84      0.881       0.88      0.933       0.81
Speed: 1.3ms preprocess, 57.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving test_results\augmented_test_predictions3\predictions.json...
Results saved to test_results\augmented_test_predictions3

Validation completed. Check the metrics above and JSON file for detailed results.


In [11]:
def evaluate_and_save_metrics(model, test_data_yaml, conf_values, iou_values, output_dir):
    
    # Initialize lists for each class
    metrics_all = []
    metrics_v = []
    metrics_c = []
    metrics_s = []

    # Iterate over all combinations of conf and IoU
    for conf in conf_values:
        for iou in iou_values:
            print(f"conf = {conf} & iou = {iou}")
            # Run evaluation
            metrics = model.val(data=test_data_yaml, conf=conf, iou=iou, plots=False)

            # Access the overall mean results by calling the method
            mean_results = metrics.mean_results()  # Call the method
            metrics_all.append({
                "iou": iou,
                "conf": conf,
                "p": mean_results[0],  # Precision
                "r": mean_results[1],  # Recall
                "map50": mean_results[2],  # mAP@50
                "map50-95": mean_results[3]  # mAP@50-95
            })

            # Access per-class metrics using names and maps
            class_maps = metrics.maps  # Per-class mAP values
            metrics_v.append({
                "iou": iou,
                "conf": conf,
                "map50": class_maps[0],  # mAP@50 for V
                "map50-95": class_maps[0]  # mAP@50-95 for V
            })
            metrics_c.append({
                "iou": iou,
                "conf": conf,
                "map50": class_maps[1],  # mAP@50 for C
                "map50-95": class_maps[1]  # mAP@50-95 for C
            })
            metrics_s.append({
                "iou": iou,
                "conf": conf,
                "map50": class_maps[2],  # mAP@50 for S
                "map50-95": class_maps[2]  # mAP@50-95 for S
            })

    # Create dataframes
    df_all = pd.DataFrame(metrics_all)
    df_v = pd.DataFrame(metrics_v)
    df_c = pd.DataFrame(metrics_c)
    df_s = pd.DataFrame(metrics_s)

    # Save dataframes to CSV files
    df_all.to_csv(f"{output_dir}/metrics_all.csv", index=False)
    df_v.to_csv(f"{output_dir}/metrics_v.csv", index=False)
    df_c.to_csv(f"{output_dir}/metrics_c.csv", index=False)
    df_s.to_csv(f"{output_dir}/metrics_s.csv", index=False)

    print("All metrics saved successfully.")

# Example usage
conf_values = [0.001, 0.1, 0.3, 0.5]
iou_values = [0.3, 0.5, 0.7, 0.9]
output_dir = "./augmented_results"

# Run the function
evaluate_and_save_metrics(model, test_data_yaml, conf_values, iou_values, output_dir)

conf = 0.001 & iou = 0.3
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:13

                   all        163        878       0.75      0.844      0.865      0.729
                     V        162        694      0.804      0.871      0.914      0.736
                     C         79        100      0.585       0.78      0.749      0.643
                     S         64         84      0.861      0.883      0.933      0.809
Speed: 1.2ms preprocess, 57.7ms inference, 0.0ms loss, 0.5ms postprocess per image
conf = 0.001 & iou = 0.5
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:19

                   all        163        878      0.761      0.839      0.865      0.729
                     V        162        694      0.817      0.867      0.914      0.735
                     C         79        100      0.587       0.77      0.748      0.643
                     S         64         84       0.88      0.881      0.932      0.808
Speed: 1.6ms preprocess, 88.8ms inference, 0.0ms loss, 0.6ms postprocess per image
conf = 0.001 & iou = 0.7
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


...

                   all        163        878      0.843      0.728      0.807      0.706
                     V        162        694       0.88      0.755       0.85      0.717
                     C         79        100      0.707       0.63      0.684      0.598
                     S         64         84      0.944      0.798      0.887      0.803
Speed: 2.3ms preprocess, 123.4ms inference, 0.0ms loss, 1.5ms postprocess per image
conf = 0.5 & iou = 0.7
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:36

                   all        163        878      0.843      0.728      0.807      0.706
                     V        162        694       0.88      0.755       0.85      0.717
                     C         79        100      0.707       0.63      0.684      0.598
                     S         64         84      0.944      0.798      0.887      0.803
Speed: 4.2ms preprocess, 152.0ms inference, 0.0ms loss, 1.0ms postprocess per image
conf = 0.5 & iou = 0.9
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:35

                   all        163        878      0.825       0.72        0.8      0.701
                     V        162        694       0.83      0.743      0.837      0.707
                     C         79        100      0.729      0.619      0.682      0.596
                     S         64         84      0.918      0.798      0.883      0.799
Speed: 4.1ms preprocess, 151.3ms inference, 0.0ms loss, 1.4ms postprocess per image
All metrics saved successfully.
